
# Computer Vision Classification Demo

This notebook demonstrates how to build a simple image classification pipeline using both **PyTorch** and **TensorFlow**. The examples use the Fashion-MNIST dataset so that you can run everything quickly on Google Colab without installing many extra libraries.



## 1. Imports and Random Seeds

We start by importing the required libraries and setting seeds for reproducibility. Both frameworks are available by default in most Colab runtimes.


In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

import tensorflow as tf

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')


## 2. PyTorch Classification Workflow

We will train a lightweight convolutional neural network (CNN) on a subset of Fashion-MNIST. Using a subset keeps the runtime short while still demonstrating the end-to-end workflow.



### 2.1 Load and Inspect the Dataset

Fashion-MNIST contains 28×28 grayscale images of clothing items. `torchvision` downloads the dataset automatically and provides convenient transforms.


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

# Use a manageable subset for faster demo runs
train_subset = Subset(train_dataset, range(6000))
test_subset = Subset(test_dataset, range(1000))

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=128)

class_names = train_dataset.classes
print(f'Train batches: {len(train_loader)}, Test batches: {len(test_loader)}')


### 2.2 Define the CNN Model

A small CNN with two convolutional layers followed by a fully connected head is sufficient for this demo.


In [ ]:
class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        return self.fc2(x)

model = FashionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


### 2.3 Train the Model

We train the network for a few epochs. Even with only one or two epochs, the accuracy should surpass random guessing and is enough for a classroom demo.


In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
    return total_loss / len(loader.dataset)

EPOCHS = 2
for epoch in range(1, EPOCHS + 1):
    loss = train(model, train_loader, optimizer, criterion)
    print(f'Epoch {epoch}/{EPOCHS} - Training Loss: {loss:.4f}')


### 2.4 Evaluate on the Test Set

We compute the accuracy on the held-out test subset to confirm that the model learned meaningful patterns.


In [ ]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

accuracy = evaluate(model, test_loader)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


### 2.5 Visualize Predictions

Finally, we inspect a few predictions to make the results more tangible for students.


In [ ]:
model.eval()
images, labels = next(iter(test_loader))
images, labels = images.to(device), labels.to(device)
outputs = model(images)
preds = outputs.argmax(dim=1)

plt.figure(figsize=(10, 4))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(images[i].cpu().squeeze(), cmap='gray')
    plt.title(f'True: {class_names[labels[i]]}
Pred: {class_names[preds[i]]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


## 3. TensorFlow Classification Workflow

Next, we recreate the same experiment using TensorFlow/Keras. This provides a side-by-side comparison of the two most popular deep learning frameworks.



### 3.1 Load and Prepare the Data

Keras ships with the Fashion-MNIST dataset. We normalize pixel values to the `[0, 1]` range and keep a validation split for monitoring training.


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Keep the dataset small for quick runs
train_limit = 6000
test_limit = 1000
x_train, y_train = x_train[:train_limit], y_train[:train_limit]
x_test, y_test = x_test[:test_limit], y_test[:test_limit]

# Normalize and add a channel dimension
x_train = (x_train / 255.0)[..., np.newaxis]
x_test = (x_test / 255.0)[..., np.newaxis]

val_split = 0.1
print(f"Training samples: {x_train.shape[0]}, Test samples: {x_test.shape[0]}")


### 3.2 Build the Keras Model

The architecture mirrors the PyTorch model: two convolutional blocks followed by dense layers.


In [ ]:
keras_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

keras_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

keras_model.summary()


### 3.3 Train the Model

We train for a few epochs with a validation split so that we can monitor both training and validation accuracy.


In [ ]:
history = keras_model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=64,
    validation_split=val_split,
    verbose=2
)


### 3.4 Evaluate and Visualize Predictions

As with the PyTorch model, we compute the test accuracy and look at a few sample predictions.


In [ ]:
test_loss, test_acc = keras_model.evaluate(x_test, y_test, verbose=0)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

probabilities = tf.nn.softmax(keras_model.predict(x_test[:6]))
predictions = tf.argmax(probabilities, axis=1)

plt.figure(figsize=(10, 4))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(x_test[i].squeeze(), cmap='gray')
    plt.title(f'True: {class_names[y_test[i]]}
Pred: {class_names[predictions[i]]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


## 4. Key Takeaways

* Both PyTorch and TensorFlow follow the same high-level workflow: prepare data, build a model, train, and evaluate.
* Subclassed models (PyTorch) and Sequential models (TensorFlow) express the same architecture with minimal syntax differences.
* Fashion-MNIST is a lightweight dataset that makes it easy to demonstrate core computer vision concepts in class.
